In [1]:
import pandas as pd
import numpy as np
import operator
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
import re

#LEER EL DATASET

df=pd.read_csv('Dataset.csv')
#L=np.asarray(df)
#print(L)

data = pd.read_csv("Dataset.csv", sep=",")
Tweet = data[["Tweet"]]
T=np.asarray(Tweet)

##########################################
punctuation = list(string.punctuation)
stemmer = SnowballStemmer("spanish")
stop = stopwords.words('spanish') + punctuation
#############Documentos##################
documentA = str(T[55])
documentB = str(T[57])
########################################
emoticons_str = r"""
    (?:
        [:=;] # :(
        [oO\-]? # No se
        [D\)\]\(\]/\\OpP] # 
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
##############Metodos####################



def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    # Romper la oración en el tokens
    tokens_first = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE )
    tokens = tokens_first.findall(s)
        
    # eliminar tokens vacíos, links, menciones, stopwords y hastags
    tokens = [token for token in tokens if (token != "" ) and (token not in stop) and (not token.startswith(('#', '@', 'https:', 'http:')))]
    #lematizar
    tokens = [stemmer.stem(token) for token in tokens]
    # print(tokens)
    # funcion zip genera n-grams
    # Concatena los tokens en los ngrams y los retorna
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict


def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

#########################################

##########Preprocesamiento##########
n=2
bagOfWordsA = generate_ngrams(documentA, n)
# print(bagOfWordsA)
bagOfWordsB = generate_ngrams(documentB, n)
# print(bagOfWordsB)


uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB))

numOfWordsA = dict.fromkeys(uniqueWords, 0)
# print(numOfWordsA)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
numOfWordsB = dict.fromkeys(uniqueWords, 0)
# print( numOfWordsA)
for word in bagOfWordsB:
    numOfWordsB[word] += 1
###################################

tfA = computeTF(numOfWordsA, bagOfWordsA)
# print(tfA)
tfB = computeTF(numOfWordsB, bagOfWordsB)
#print(tfB)              
idfs = computeIDF([numOfWordsA, numOfWordsB])

                
                
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
print(tfidfA)
df = pd.DataFrame([tfidfA, tfidfB])
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(df)
                



{'pued tard': 0.0, 'justici reparacion': 0.028881132523331052, 'd proces': 0.028881132523331052, 'justici transicional': 0.0, 'derech d': 0.028881132523331052, 'victim d': 0.028881132523331052, 'victim verd': 0.028881132523331052, 'pod congres': 0.028881132523331052, 'ordinari si': 0.0, 'anos d': 0.028881132523331052, 'proces pued': 0.0, 'proces q': 0.028881132523331052, 'defiend dar': 0.028881132523331052, 'calvari victim': 0.028881132523331052, 'd calvari': 0.028881132523331052, 'justici ordinari': 0.0, 'verd justici': 0.028881132523331052, 'dar 3': 0.028881132523331052, 'firm d': 0.028881132523331052, '3 anos': 0.028881132523331052, 'congres pod': 0.028881132523331052, 'd dej': 0.028881132523331052, 'pod mont': 0.028881132523331052, 'vulner derech': 0.028881132523331052, 'd victim': 0.028881132523331052, 'q vulner': 0.028881132523331052, 'dej band': 0.028881132523331052, 'nivel impun': 0.0, 'impun altos': 0.0, 'altos proces': 0.0, 'anos comparacion': 0.0, 'mont defiend': 0.028881132